In [ ]:
from jupyter_dash import JupyterDash


In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go


import pandas as pd

import pickle


In [ ]:
with open('price_dict.pkl', 'rb') as f:
    price_dict = pickle.load(f)

with open('si_dict.pkl', 'rb') as f:
    si_dict = pickle.load(f)

with open('ibd_dict.pkl', 'rb') as f:
    ibd_dict = pickle.load(f)


In [ ]:
tickers = list(price_dict.keys())


In [ ]:
layout = {
    'margin': {'b': 15, 't': 15},
    'xaxis': {'domain': [0, 0.9]},
    'yaxis': {'tickfont': {'color': '#636efa'}, 'title': {'font': {'color': '#636efa'}, 'text': 'Price'}},
    'yaxis2': {'anchor': 'x',
               'overlaying': 'y',
               'side': 'right',
               'tickfont': {'color': '#ef553b'},
               'title': {'font': {'color': '#ef553b'}, 'text': 'Days to cover'}},
    'yaxis3': {'anchor': 'free',
               'overlaying': 'y',
               'position': 0.95,
               'side': 'right',
               'tickfont': {'color': '#00cc96'},
               'title': {'font': {'color': '#00cc96'}, 'text': 'Borrowing fee'}}
}


In [ ]:
fig_graph = go.Figure()
fig_graph.update_layout(**layout)

fig_graph_pct_change = go.Figure()
fig_graph_pct_change.update_layout(**layout)


In [ ]:
app = JupyterDash(__name__)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div(style={'background': 'white', 'font-family': "-apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif"},
                      children=[
    html.Div([
        dcc.Dropdown(
            id="ticker-select",
            options=[{'label': ticker, 'value': ticker} for ticker in tickers],
            value=tickers[0]
        )
    ], style={'width': '30%', 'margin': 'auto'}),
    html.H2(children='Data', style={'margin': 0}),
    html.Div([
        dcc.Graph(
            id='graph',
        )
    ], style={}),
    html.H2(children='Percent change', style={'margin': 0}),
    html.Div([
        dcc.Graph(
            id='graph_pct_change',
        )
    ], style={})
])


@app.callback(
    Output('graph', 'figure'),
    Output('graph_pct_change', 'figure'),
    Input('ticker-select', 'value')
)
def update_figure(ticker):
    fig_graph.data = []

    fig_graph.add_trace(go.Scatter(
        x=price_dict[ticker].index,
        y=price_dict[ticker]['Adj Close'],
        name="Adj Close"
    ))

    fig_graph.add_trace(go.Scatter(
        x=si_dict[ticker].index,
        y=si_dict[ticker]['daysToCover'],
        name="Short Interest ratio",
        yaxis='y2',
        mode='lines+markers',
    ))

    fig_graph.add_trace(go.Scatter(
        x=ibd_dict[ticker].index,
        y=ibd_dict[ticker]['fee'],
        name="Borrowing fee",
        yaxis='y3'
    ))

    fig_graph_pct_change.data = []

    fig_graph_pct_change.add_trace(go.Scatter(
        x=price_dict[ticker].index,
        y=price_dict[ticker]['Adj Close'].pct_change(),
        name="Adj Close"
    ))

    fig_graph_pct_change.add_trace(go.Scatter(
        x=si_dict[ticker].index,
        y=si_dict[ticker]['daysToCover'].pct_change(),
        name="Short Interest ratio",
        yaxis='y2',
        mode='lines+markers',
    ))

    fig_graph_pct_change.add_trace(go.Scatter(
        x=ibd_dict[ticker].index,
        y=ibd_dict[ticker]['fee'].pct_change(),
        name="Borrowing fee",
        yaxis='y3'
    ))

    return fig_graph, fig_graph_pct_change


In [ ]:
app.run_server(mode='inline') # use mode='external' to run on localhost:8050